# Lung Cancer Assistant (LCA) - Complete System Coverage

This notebook provides **complete coverage** of the Lung Cancer Assistant system:

## Table of Contents

1. **Setup & Configuration**
2. **SNOMED-CT Ontology Integration**
3. **LUCADA Ontology Creation**
4. **Clinical Guideline Rules Engine (NICE CG121)**
5. **6-Agent Workflow Architecture**
   - 5.1 IngestionAgent
   - 5.2 SemanticMappingAgent
   - 5.3 ClassificationAgent
   - 5.4 ConflictResolutionAgent
   - 5.5 PersistenceAgent
   - 5.6 ExplanationAgent
6. **Complete Workflow Execution**
7. **Neo4j Database Integration**
8. **MCP Server Tools**
9. **Synthetic Patient Generation**
10. **Batch Processing & Cohort Analysis**
11. **REST API Integration**
12. **Performance Benchmarks**

---

**Based on:** Sesen et al., "Lung Cancer Assistant: An Ontology-Driven Clinical Decision Support System", University of Oxford

**Architecture:** 6-Agent LangGraph Workflow with OWL Ontology and Neo4j Knowledge Graph

**CRITICAL PRINCIPLE:** "Neo4j as a tool, not a brain" - All medical reasoning in Python/OWL

---
## Part 1: Setup & Configuration

In [ ]:
# Setup and Imports
import sys
import os
import json
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, Any, List

# Add backend to path
backend_path = Path.cwd().parent / 'backend'
sys.path.insert(0, str(backend_path))

import warnings
warnings.filterwarnings('ignore')

# Environment configuration
os.environ["OLLAMA_BASE_URL"] = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
os.environ["OLLAMA_MODEL"] = os.getenv("OLLAMA_MODEL", "llama3.2:latest")

print("=" * 80)
print("LUNG CANCER ASSISTANT - COMPLETE SYSTEM COVERAGE")
print("=" * 80)
print(f"Backend Path: {backend_path}")
print(f"Ollama URL: {os.environ['OLLAMA_BASE_URL']}")
print(f"Ollama Model: {os.environ['OLLAMA_MODEL']}")
print(f"Timestamp: {datetime.now().isoformat()}")
print("✓ Environment setup complete")

---
## Part 2: SNOMED-CT Ontology Integration

SNOMED-CT (Systematized Nomenclature of Medicine - Clinical Terms) provides a comprehensive clinical terminology with 350K+ concepts.

The SNOMEDLoader maps clinical concepts to standardized SNOMED-CT codes for interoperability.

In [ ]:
from src.ontology.snomed_loader import SNOMEDLoader

# Initialize SNOMED loader
snomed_path = "../ontology-2026-01-17_12-36-08.owl"
snomed = SNOMEDLoader(snomed_path)

print("SNOMED-CT Ontology Loader")
print("=" * 80)
print(f"Ontology path: {snomed.owl_path}")
print(f"File exists: {Path(snomed.owl_path).exists()}")

print("\n✓ SNOMED loader initialized")
print("\nNote: Full loading is memory-intensive (350K+ concepts)")
print("Using pre-defined lung cancer concept mappings.")

In [ ]:
# Display all SNOMED-CT mapping dictionaries
print("SNOMED-CT Concept Mappings for Lung Cancer")
print("=" * 80)

# Histology mappings
print(f"\n1. HISTOLOGY TYPES ({len(snomed.HISTOLOGY_MAP)} mappings):")
print("-" * 40)
for hist, code in snomed.HISTOLOGY_MAP.items():
    print(f"   {hist}: SCTID {code}")

# TNM Stage mappings
print(f"\n2. TNM STAGES ({len(snomed.STAGE_MAP)} mappings):")
print("-" * 40)
for stage, code in snomed.STAGE_MAP.items():
    print(f"   Stage {stage}: SCTID {code}")

# Treatment mappings
print(f"\n3. TREATMENTS ({len(snomed.TREATMENT_MAP)} mappings):")
print("-" * 40)
for treatment, code in snomed.TREATMENT_MAP.items():
    print(f"   {treatment}: SCTID {code}")

# Performance Status mappings
print(f"\n4. WHO PERFORMANCE STATUS ({len(snomed.PERFORMANCE_STATUS_MAP)} mappings):")
print("-" * 40)
for ps, code in snomed.PERFORMANCE_STATUS_MAP.items():
    print(f"   WHO Grade {ps}: SCTID {code}")

# Outcome mappings
print(f"\n5. TREATMENT OUTCOMES ({len(snomed.OUTCOME_MAP)} mappings):")
print("-" * 40)
for outcome, code in snomed.OUTCOME_MAP.items():
    print(f"   {outcome}: SCTID {code}")

print("\n" + "=" * 80)
print("✓ SNOMED-CT mappings loaded successfully")

In [ ]:
# Test SNOMED mapping with a patient
print("SNOMED-CT Patient Mapping Test")
print("=" * 80)

test_patient = {
    "patient_id": "SNOMED_TEST_001",
    "histology_type": "Adenocarcinoma",
    "tnm_stage": "IIIA",
    "performance_status": 1,
    "laterality": "Right"
}

print(f"\nInput Patient Data:")
print(json.dumps(test_patient, indent=2))

# Map patient to SNOMED codes
snomed_codes = snomed.map_patient_to_snomed(test_patient)

print(f"\nSNOMED-CT Mappings:")
for key, code in snomed_codes.items():
    print(f"   {key}: SCTID {code}")

# Generate OWL expression
owl_expr = snomed.generate_owl_expression(test_patient)
print(f"\nOWL 2 Class Expression (Manchester Syntax):")
print("-" * 80)
print(owl_expr)

print("\n✓ SNOMED patient mapping complete")

---
## Part 3: LUCADA Ontology Creation

LUCADA (LUng CAncer Decision Assistant) is the domain ontology implementing:
- 60+ OWL classes
- 35+ object properties
- 60+ data properties
- SNOMED-CT integration
- Patient scenario classification

In [ ]:
from src.ontology.lucada_ontology import LUCADAOntology

# Create LUCADA ontology
print("Creating LUCADA Ontology...")
print("=" * 80)

lucada = LUCADAOntology()
onto = lucada.create()

print(f"\nOntology Statistics:")
print(f"   IRI: {onto.base_iri}")
print(f"   Classes: {len(list(onto.classes()))}")
print(f"   Object Properties: {len(list(onto.object_properties()))}")
print(f"   Data Properties: {len(list(onto.data_properties()))}")
print(f"   Individuals: {len(list(onto.individuals()))}")

# Show key domain classes
print("\nKey Domain Classes:")
key_classes = ["Patient", "ClinicalFinding", "Histology", "TreatmentPlan", 
               "PatientScenario", "Argument", "Decision", "PerformanceStatus"]
for cls in key_classes:
    if hasattr(onto, cls):
        print(f"   ✓ {cls}")
    else:
        print(f"   ✗ {cls} (not found)")

print("\n" + "=" * 80)
print("✓ LUCADA ontology created successfully")

In [ ]:
# Explore LUCADA ontology structure
print("LUCADA Ontology Class Hierarchy")
print("=" * 80)

# List all classes by category
all_classes = list(onto.classes())

categories = {
    "Patient Domain": [c for c in all_classes if "Patient" in str(c) or "Person" in str(c)],
    "Clinical Findings": [c for c in all_classes if "Finding" in str(c) or "Histology" in str(c) or "Stage" in str(c)],
    "Treatments": [c for c in all_classes if "Treatment" in str(c) or "Therapy" in str(c) or "Surgery" in str(c)],
    "Performance Status": [c for c in all_classes if "Performance" in str(c) or "WHO" in str(c)],
    "Argumentation": [c for c in all_classes if "Argument" in str(c) or "Decision" in str(c) or "Scenario" in str(c)],
}

for category, classes in categories.items():
    if classes:
        print(f"\n{category}:")
        for cls in classes[:10]:  # Limit to 10 per category
            print(f"   - {cls.name}")
        if len(classes) > 10:
            print(f"   ... and {len(classes) - 10} more")

print("\n" + "=" * 80)
print(f"Total Classes: {len(all_classes)}")

In [ ]:
# Explore ontology properties
print("LUCADA Ontology Properties")
print("=" * 80)

# Object properties (relationships)
obj_props = list(onto.object_properties())
print(f"\nObject Properties ({len(obj_props)} total):")
print("-" * 40)
for prop in obj_props[:15]:
    print(f"   {prop.name}")
if len(obj_props) > 15:
    print(f"   ... and {len(obj_props) - 15} more")

# Data properties (attributes)
data_props = list(onto.data_properties())
print(f"\nData Properties ({len(data_props)} total):")
print("-" * 40)
for prop in data_props[:15]:
    print(f"   {prop.name}")
if len(data_props) > 15:
    print(f"   ... and {len(data_props) - 15} more")

print("\n" + "=" * 80)

In [ ]:
# Save ontology to file
output_path = "../output/lucada_ontology.owl"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

lucada.save(output_path)
print(f"✓ Ontology saved to: {output_path}")
print(f"   File size: {os.path.getsize(output_path) / 1024:.1f} KB")

---
## Part 4: Clinical Guideline Rules Engine (NICE CG121)

The GuidelineRuleEngine implements 7 formalized NICE CG121 rules for lung cancer treatment:
- R1: Chemotherapy for Stage III-IV NSCLC
- R2: Surgery for Stage I-II NSCLC
- R3: Radiotherapy for Stage I-IIIA NSCLC
- R4: Palliative care for advanced/poor PS
- R5: Chemotherapy for SCLC
- R6: Chemoradiotherapy for Stage IIIA/IIIB
- R7: Immunotherapy for advanced NSCLC with PD-L1

In [ ]:
from src.ontology.guideline_rules import GuidelineRuleEngine

# Initialize rule engine with LUCADA ontology
rule_engine = GuidelineRuleEngine(lucada)

print("Clinical Guideline Rules Engine")
print("=" * 80)
print(f"Source: NICE Clinical Guideline CG121")
print(f"Total Rules: {len(rule_engine.rules)}")
print("\n" + "-" * 80)

# Display all rules with details
for rule in rule_engine.get_all_rules():
    print(f"\n{rule.rule_id}: {rule.name}")
    print(f"   Source: {rule.source}")
    print(f"   Treatment: {rule.recommended_treatment}")
    print(f"   Intent: {rule.treatment_intent}")
    print(f"   Evidence Level: {rule.evidence_level}")
    print(f"   Priority: {rule.priority}/100")
    print(f"   Description: {rule.description[:100]}..." if len(rule.description) > 100 else f"   Description: {rule.description}")

print("\n" + "=" * 80)
print("✓ Guideline rules loaded")

In [ ]:
# Test rule matching with different patient profiles
print("Guideline Rule Matching Tests")
print("=" * 80)

test_profiles = [
    {
        "name": "Early Stage NSCLC (Operable)",
        "patient_id": "TEST_EARLY_001",
        "age": 62,
        "sex": "F",
        "tnm_stage": "IB",
        "histology_type": "Adenocarcinoma",
        "performance_status": 0,
        "fev1_percent": 85.0
    },
    {
        "name": "Locally Advanced NSCLC",
        "patient_id": "TEST_ADVANCED_001",
        "age": 68,
        "sex": "M",
        "tnm_stage": "IIIA",
        "histology_type": "SquamousCellCarcinoma",
        "performance_status": 1,
        "fev1_percent": 65.0
    },
    {
        "name": "Metastatic NSCLC (Good PS)",
        "patient_id": "TEST_METASTATIC_001",
        "age": 71,
        "sex": "M",
        "tnm_stage": "IV",
        "histology_type": "Adenocarcinoma",
        "performance_status": 1,
        "fev1_percent": 55.0
    },
    {
        "name": "Small Cell Lung Cancer",
        "patient_id": "TEST_SCLC_001",
        "age": 65,
        "sex": "M",
        "tnm_stage": "IIIB",
        "histology_type": "SmallCellCarcinoma",
        "performance_status": 1,
        "fev1_percent": 60.0
    },
    {
        "name": "Poor Performance Status",
        "patient_id": "TEST_POOR_PS_001",
        "age": 78,
        "sex": "F",
        "tnm_stage": "IV",
        "histology_type": "Adenocarcinoma",
        "performance_status": 3,
        "fev1_percent": 40.0
    }
]

for profile in test_profiles:
    print(f"\n{profile['name']}")
    print(f"   Stage: {profile['tnm_stage']}, Histology: {profile['histology_type']}, PS: {profile['performance_status']}")
    
    recommendations = rule_engine.classify_patient(profile)
    print(f"   ➜ {len(recommendations)} matching guideline(s):")
    
    for rec in recommendations[:3]:  # Show top 3
        print(f"      • {rec['recommended_treatment']} ({rec['evidence_level']})")
        print(f"        Rule: {rec['rule_id']}, Priority: {rec['priority']}/100")

print("\n" + "=" * 80)

---
## Part 5: 6-Agent Workflow Architecture

The LCA system uses a 6-agent LangGraph workflow:

```
┌─────────────────────────────────────────────────────────────────┐
│                    WORKFLOW ORCHESTRATOR                        │
│                   (LangGraph Implementation)                    │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  [1] IngestionAgent     → [2] SemanticMappingAgent             │
│      (validate/normalize)     (SNOMED-CT mapping)              │
│         ↓                         ↓                            │
│  [3] ClassificationAgent → [4] ConflictResolutionAgent         │
│      (ontology + NICE rules)    (rank/deduplicate)             │
│         ↓                         ↓                            │
│  [5] PersistenceAgent   → [6] ExplanationAgent                 │
│      (ONLY WRITES TO NEO4J)     (MDT summary)                  │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**CRITICAL PRINCIPLE:** "Neo4j as a tool, not a brain"
- All medical reasoning happens in Python/OWL
- Neo4j is only for storage and retrieval
- Only PersistenceAgent writes to Neo4j

In [ ]:
# Import all 6 agents
from src.agents.ingestion_agent import IngestionAgent
from src.agents.semantic_mapping_agent import SemanticMappingAgent
from src.agents.classification_agent import ClassificationAgent, PatientScenario
from src.agents.conflict_resolution_agent import ConflictResolutionAgent
from src.agents.explanation_agent import ExplanationAgent
from src.agents.lca_workflow import LCAWorkflow, analyze_patient

print("6-Agent Architecture Loaded")
print("=" * 80)
print("\n   [1] IngestionAgent       - Validates and normalizes raw patient data")
print("   [2] SemanticMappingAgent - Maps clinical concepts to SNOMED-CT codes")
print("   [3] ClassificationAgent  - Applies LUCADA ontology and NICE guidelines")
print("   [4] ConflictResolutionAgent - Resolves conflicting recommendations")
print("   [5] PersistenceAgent     - THE ONLY AGENT THAT WRITES TO NEO4J")
print("   [6] ExplanationAgent     - Generates MDT summaries")
print("\n" + "=" * 80)
print("CRITICAL: 'Neo4j as a tool, not a brain'")

### 5.1 IngestionAgent

Validates and normalizes raw patient data:
- Validates required fields (tnm_stage, histology_type)
- Normalizes TNM staging (e.g., "Stage IIA" → "IIA")
- Normalizes histology types to standard enums
- Returns PatientFact or validation errors

In [ ]:
# Test IngestionAgent with Jenny_Sesen from original paper (Figure 2)
print("Agent 1: IngestionAgent")
print("=" * 80)

# Jenny Sesen from original LCA paper
jenny_sesen = {
    "patient_id": "200312",
    "name": "Jenny_Sesen",
    "sex": "Female",
    "age_at_diagnosis": 72,
    "tnm_stage": "IIA",  # Stage IIA
    "histology_type": "Carcinosarcoma",  # Subtype of NSCLC
    "performance_status": 1,
    "laterality": "Right",
    "diagnosis": "Malignant Neoplasm of Lung"
}

print("\nInput Patient Data (from original paper Figure 2):")
print(json.dumps(jenny_sesen, indent=2))

# Run IngestionAgent
ingestion_agent = IngestionAgent()
patient_fact, errors = ingestion_agent.execute(jenny_sesen)

print("\n" + "-" * 80)
if patient_fact:
    print("✓ Validation successful!")
    print(f"\nNormalized PatientFact:")
    print(f"   Patient ID: {patient_fact.patient_id}")
    print(f"   Name: {patient_fact.name}")
    print(f"   Sex: {patient_fact.sex}")
    print(f"   Age: {patient_fact.age_at_diagnosis}")
    print(f"   TNM Stage: {patient_fact.tnm_stage}")
    print(f"   Histology: {patient_fact.histology_type}")
    print(f"   Performance Status: WHO Grade {patient_fact.performance_status}")
    print(f"   Laterality: {patient_fact.laterality}")
else:
    print("✗ Validation failed!")
    for error in errors:
        print(f"   Error: {error}")

print("\n" + "=" * 80)

In [ ]:
# Test validation error handling
print("IngestionAgent - Validation Error Handling")
print("=" * 80)

invalid_patients = [
    {"patient_id": "INVALID_001", "name": "Missing Stage", "histology_type": "Adenocarcinoma"},
    {"patient_id": "INVALID_002", "name": "Invalid Stage", "tnm_stage": "Stage 5", "histology_type": "Adenocarcinoma"},
    {"patient_id": "INVALID_003", "name": "Invalid PS", "tnm_stage": "IIA", "histology_type": "Adenocarcinoma", "performance_status": 5},
]

for patient in invalid_patients:
    print(f"\nTesting: {patient['name']}")
    fact, errs = ingestion_agent.execute(patient)
    if errs:
        for err in errs:
            print(f"   ✗ {err}")
    else:
        print(f"   ✓ Valid (unexpected)")

print("\n" + "=" * 80)
print("✓ Validation error handling works correctly")

### 5.2 SemanticMappingAgent

Maps clinical concepts to SNOMED-CT codes:
- Diagnosis → SNOMED diagnosis code
- Histology → SNOMED histology code
- TNM Stage → SNOMED stage code
- Performance Status → SNOMED PS code
- Returns PatientFactWithCodes + confidence score

In [ ]:
# Test SemanticMappingAgent
print("Agent 2: SemanticMappingAgent")
print("=" * 80)

semantic_agent = SemanticMappingAgent()
patient_with_codes, mapping_confidence = semantic_agent.execute(patient_fact)

print(f"\n✓ SNOMED-CT mapping completed")
print(f"   Mapping Confidence: {mapping_confidence:.2%}")

print(f"\nSNOMED-CT Codes:")
print(f"   Diagnosis:          SCTID {patient_with_codes.snomed_diagnosis_code}")
print(f"   Histology:          SCTID {patient_with_codes.snomed_histology_code}")
print(f"   Stage:              SCTID {patient_with_codes.snomed_stage_code}")
print(f"   Performance Status: SCTID {patient_with_codes.snomed_ps_code}")
print(f"   Laterality:         SCTID {patient_with_codes.snomed_laterality_code}")

# Check NSCLC subtype
is_nsclc = semantic_agent.is_nsclc_subtype(patient_with_codes.snomed_histology_code or "")
print(f"\n   Is NSCLC subtype: {is_nsclc}")
print(f"   Note: Carcinosarcoma is a subtype of Non-Small Cell Lung Carcinoma")

print("\n" + "=" * 80)

### 5.3 ClassificationAgent

Applies LUCADA ontology and NICE guidelines:
- Determines patient scenario (8 possible scenarios)
- Matches applicable guideline rules
- Returns ranked treatment recommendations with evidence levels

In [ ]:
# Test ClassificationAgent
print("Agent 3: ClassificationAgent")
print("=" * 80)

classification_agent = ClassificationAgent()
classification = classification_agent.execute(patient_with_codes)

print(f"\n✓ Classification completed!")
print(f"\nPatient Scenario: {classification.scenario}")
print(f"Confidence: {classification.scenario_confidence:.2%}")

print(f"\nReasoning Chain:")
for i, step in enumerate(classification.reasoning_chain, 1):
    print(f"   {i}. {step}")

print(f"\nTreatment Recommendations:")
for rec in classification.recommendations:
    if isinstance(rec, dict):
        print(f"\n   Rank {rec.get('rank', '?')}: {rec.get('treatment', 'Unknown')}")
        print(f"      Evidence Level: {rec.get('evidence_level', 'N/A')}")
        print(f"      Intent: {rec.get('intent', 'N/A')}")
        print(f"      Guideline: {rec.get('guideline_reference', 'N/A')}")
        rationale = rec.get('rationale', '')
        print(f"      Rationale: {rationale[:80]}..." if len(rationale) > 80 else f"      Rationale: {rationale}")
    else:
        print(f"\n   Rank {rec.rank}: {rec.treatment}")
        print(f"      Evidence Level: {rec.evidence_level.value}")
        print(f"      Intent: {rec.intent.value if rec.intent else 'N/A'}")
        print(f"      Guideline: {rec.guideline_reference}")
        print(f"      Rationale: {rec.rationale[:80]}..." if len(rec.rationale) > 80 else f"      Rationale: {rec.rationale}")

print("\n" + "=" * 80)

### 5.4 ConflictResolutionAgent

Resolves conflicting recommendations:
- Detects conflicts between treatment options
- Applies evidence hierarchy (Grade A > B > C > D)
- Returns ranked, deduplicated recommendations

In [ ]:
# Test ConflictResolutionAgent
print("Agent 4: ConflictResolutionAgent")
print("=" * 80)

conflict_agent = ConflictResolutionAgent()
resolved_classification, conflict_reports = conflict_agent.execute(classification)

print(f"\n✓ Conflict resolution completed!")
print(f"\nConflicts Detected: {len(conflict_reports)}")

if conflict_reports:
    for report in conflict_reports:
        print(f"\n   Conflict Type: {report.conflict_type}")
        print(f"   Original Options: {', '.join(report.original_recommendations)}")
        print(f"   Resolution: {report.resolution}")
        print(f"   Rationale: {report.rationale}")
else:
    print("   No conflicts detected - recommendations are consistent")

print(f"\nFinal Ranked Recommendations:")
for rec in resolved_classification.recommendations:
    if isinstance(rec, dict):
        print(f"   {rec.get('rank', '?')}. {rec.get('treatment', 'Unknown')} ({rec.get('evidence_level', 'N/A')})")
    else:
        print(f"   {rec.rank}. {rec.treatment} ({rec.evidence_level.value})")

print("\n" + "=" * 80)

### 5.5 PersistenceAgent

**CRITICAL: This is the ONLY agent that writes to Neo4j.**

Functions:
- Saves patient facts with audit trail
- Saves inference results with provenance
- Maintains versioning and timestamps
- Returns write receipts for confirmation

Note: This section demonstrates the agent interface. Neo4j must be running for actual persistence.

In [ ]:
# Demonstrate PersistenceAgent (skipped if Neo4j not available)
print("Agent 5: PersistenceAgent")
print("=" * 80)
print("\nCRITICAL: This is the ONLY agent that writes to Neo4j.")
print("\nNeo4j Write Operations:")
print("   • save_patient_facts() - Store patient data with audit trail")
print("   • save_inference_result() - Store classification results")
print("   • mark_inference_obsolete() - Version control")
print("   • save_treatment_recommendation() - Store recommendations")

print("\nNeo4j Access Pattern:")
print("   ┌──────────────────────────┐")
print("   │  READ-ONLY Access        │")
print("   │  ────────────────────    │")
print("   │  • IngestionAgent        │")
print("   │  • SemanticMappingAgent  │")
print("   │  • ClassificationAgent   │")
print("   │  • ConflictResAgent      │")
print("   │  • ExplanationAgent      │")
print("   └──────────────────────────┘")
print("   ┌──────────────────────────┐")
print("   │  WRITE Access            │")
print("   │  ────────────────────    │")
print("   │  • PersistenceAgent ONLY │")
print("   └──────────────────────────┘")

# Check Neo4j availability
from src.db.neo4j_tools import Neo4jWriteTools
try:
    write_tools = Neo4jWriteTools()
    if write_tools.is_available:
        print("\n✓ Neo4j is available for persistence")
    else:
        print("\n⚠ Neo4j not available - persistence will be skipped")
    write_tools.close()
except Exception as e:
    print(f"\n⚠ Neo4j connection error: {e}")

print("\n" + "=" * 80)

### 5.6 ExplanationAgent

Generates MDT (Multi-Disciplinary Team) summaries:
- Clinical summary paragraphs
- Formatted recommendations
- Key considerations for discussion
- Discussion points for MDT meetings
- SNOMED mappings for audit trail
- Guideline references

In [ ]:
# Test ExplanationAgent
print("Agent 6: ExplanationAgent")
print("=" * 80)

explanation_agent = ExplanationAgent()
mdt_summary = explanation_agent.execute(patient_with_codes, resolved_classification)

print(f"\n✓ MDT Summary Generated!")
print(f"   Generated at: {mdt_summary.generated_at}")

print("\n" + "-" * 80)
print("CLINICAL SUMMARY")
print("-" * 80)
print(mdt_summary.clinical_summary)

print("\n" + "-" * 80)
print("REASONING EXPLANATION")
print("-" * 80)
print(mdt_summary.reasoning_explanation)

print("\n" + "-" * 80)
print("KEY CONSIDERATIONS")
print("-" * 80)
for consideration in mdt_summary.key_considerations:
    print(f"   • {consideration}")

print("\n" + "-" * 80)
print("DISCUSSION POINTS FOR MDT")
print("-" * 80)
for point in mdt_summary.discussion_points:
    print(f"   • {point}")

print("\n" + "-" * 80)
print("SNOMED-CT MAPPINGS (for audit)")
print("-" * 80)
for key, code in mdt_summary.snomed_mappings.items():
    print(f"   {key}: SCTID {code}")

print("\n" + "-" * 80)
print("GUIDELINE REFERENCES")
print("-" * 80)
for ref in mdt_summary.guideline_references:
    print(f"   • {ref}")

print("\n" + "-" * 80)
print("DISCLAIMER")
print("-" * 80)
print(mdt_summary.disclaimer[:200] + "..." if len(mdt_summary.disclaimer) > 200 else mdt_summary.disclaimer)

print("\n" + "=" * 80)

---
## Part 6: Complete Workflow Execution

Run the complete 6-agent workflow using the `analyze_patient` convenience function.

In [ ]:
# Complete Workflow Execution
print("Complete 6-Agent Workflow Execution")
print("=" * 80)

# Test with multiple patient scenarios
test_patients = [
    {
        "patient_id": "WORKFLOW-001",
        "name": "Early_Stage_Patient",
        "sex": "Female",
        "age_at_diagnosis": 62,
        "tnm_stage": "IB",
        "histology_type": "Adenocarcinoma",
        "performance_status": 0,
        "laterality": "Right"
    },
    {
        "patient_id": "WORKFLOW-002",
        "name": "Metastatic_Patient",
        "sex": "Male",
        "age_at_diagnosis": 65,
        "tnm_stage": "IV",
        "histology_type": "Adenocarcinoma",
        "performance_status": 1,
        "laterality": "Left"
    },
    {
        "patient_id": "WORKFLOW-003",
        "name": "SCLC_Patient",
        "sex": "Male",
        "age_at_diagnosis": 68,
        "tnm_stage": "IIIA",
        "histology_type": "SmallCellCarcinoma",
        "performance_status": 1,
        "laterality": "Right"
    }
]

for patient in test_patients:
    print(f"\n{'='*80}")
    print(f"Patient: {patient['name']} ({patient['patient_id']})")
    print(f"Stage: {patient['tnm_stage']}, Histology: {patient['histology_type']}, PS: {patient['performance_status']}")
    print("-" * 80)
    
    # Run workflow
    result = analyze_patient(patient, persist=False)
    
    print(f"\n   Workflow Status: {result.workflow_status}")
    print(f"   Processing Time: {result.processing_time_seconds:.3f} seconds")
    print(f"   Agent Chain: {' → '.join(result.agent_chain)}")
    print(f"\n   Scenario: {result.scenario}")
    print(f"   Confidence: {result.scenario_confidence:.2%}")
    
    print(f"\n   Top Recommendations:")
    for rec in result.recommendations[:3]:
        print(f"      • {rec['treatment']} ({rec['evidence_level']})")
    
    if result.errors:
        print(f"\n   Errors: {result.errors}")

print("\n" + "=" * 80)
print("✓ Complete workflow execution finished")

---
## Part 7: Neo4j Database Integration

Demonstrates the Neo4j tools with strict read/write separation.

In [ ]:
from src.db.neo4j_tools import Neo4jReadTools, Neo4jWriteTools, Neo4jTools

print("Neo4j Database Integration")
print("=" * 80)

# Check Neo4j availability
read_tools = Neo4jReadTools()

if read_tools.is_available:
    print("\n✓ Neo4j connection established")
    print(f"   URI: {read_tools.uri}")
    print(f"   Database: {read_tools.database}")
    
    print("\nAvailable READ Operations:")
    print("   • get_patient(patient_id) - Retrieve patient by ID")
    print("   • find_similar_patients(patient_fact) - Find similar cases")
    print("   • get_cohort_statistics(criteria) - Aggregate stats")
    print("   • get_historical_inferences(patient_id) - Audit trail")
    print("   • get_guideline_outcomes(guideline_id) - Rule statistics")
    print("   • get_treatment_statistics(treatment_type) - Treatment stats")
    
    print("\nAvailable WRITE Operations (PersistenceAgent ONLY):")
    print("   • save_patient_facts(patient_fact) - Store patient")
    print("   • save_inference_result(inference) - Store results")
    print("   • mark_inference_obsolete(patient_id) - Version control")
    print("   • save_treatment_recommendation(...) - Store recommendations")
    
    read_tools.close()
else:
    print("\n⚠ Neo4j not available")
    print("\nTo enable Neo4j:")
    print("   1. Start Neo4j: docker run -p 7474:7474 -p 7687:7687 neo4j:5.15-community")
    print("   2. Set environment variables:")
    print("      NEO4J_URI=bolt://localhost:7687")
    print("      NEO4J_USER=neo4j")
    print("      NEO4J_PASSWORD=password")

print("\n" + "=" * 80)

In [ ]:
# Display Neo4j schema (if available)
from src.db.neo4j_schema import LUCADAGraphDB

print("Neo4j Schema for LUCADA")
print("=" * 80)

print("\nNode Labels:")
print("   • Patient - Patient demographics and clinical data")
print("   • ClinicalFinding - Diagnosis information")
print("   • Histology - Histology type classification")
print("   • TreatmentPlan - Treatment recommendations")
print("   • Outcome - Treatment outcomes")
print("   • Inference - Workflow audit trail")
print("   • GuidelineRule - Clinical guideline rules")

print("\nRelationship Types:")
print("   • (Patient)-[:HAS_CLINICAL_FINDING]->(ClinicalFinding)")
print("   • (ClinicalFinding)-[:HAS_HISTOLOGY]->(Histology)")
print("   • (Patient)-[:RECEIVED_TREATMENT]->(TreatmentPlan)")
print("   • (TreatmentPlan)-[:HAS_OUTCOME]->(Outcome)")
print("   • (Patient)-[:HAS_INFERENCE]->(Inference)")
print("   • (GuidelineRule)-[:RECOMMENDS]->(TreatmentPlan)")

print("\nConstraints:")
print("   • Patient.patient_id IS UNIQUE")
print("   • Inference.inference_id IS UNIQUE")
print("   • GuidelineRule.rule_id IS UNIQUE")

print("\nIndexes:")
print("   • Patient(tnm_stage)")
print("   • Patient(histology_type)")
print("   • Inference(status)")
print("   • Full-text: Patient(name, notes)")

print("\n" + "=" * 80)

---
## Part 8: MCP Server Tools

The MCP (Model Context Protocol) server exposes 18 tools for external integration.

In [ ]:
print("MCP Server Tools")
print("=" * 80)

tools = [
    # Core Tools
    ("create_patient", "Create a new patient in the LUCADA ontology"),
    ("classify_patient", "Classify patient and identify applicable guidelines"),
    ("generate_recommendations", "Generate detailed AI-powered treatment recommendations"),
    ("list_guidelines", "List all available clinical guideline rules"),
    ("query_ontology", "Query the LUCADA ontology for concepts"),
    ("get_ontology_stats", "Get ontology statistics"),
    
    # SNOMED Tools
    ("search_snomed", "Search for SNOMED-CT concepts by term"),
    ("get_snomed_concept", "Get detailed SNOMED-CT concept information"),
    ("map_patient_to_snomed", "Map patient data to SNOMED-CT codes"),
    ("generate_owl_expression", "Generate OWL 2 class expression"),
    ("get_lung_cancer_concepts", "Get all lung cancer SNOMED-CT concepts"),
    
    # 6-Agent Workflow Tools
    ("run_6agent_workflow", "Run complete 6-agent workflow"),
    ("get_workflow_info", "Get workflow architecture information"),
    ("run_ingestion_agent", "Run only the IngestionAgent"),
    ("run_semantic_mapping_agent", "Run only the SemanticMappingAgent"),
    ("run_classification_agent", "Run only the ClassificationAgent"),
    ("generate_mdt_summary", "Generate MDT summary"),
    ("validate_patient_schema", "Validate patient data against schema"),
]

print(f"\nTotal Tools: {len(tools)}")
print("\n" + "-" * 80)

for i, (name, desc) in enumerate(tools, 1):
    print(f"   {i:2}. {name}")
    print(f"       {desc}")

print("\n" + "-" * 80)
print("\nMCP Server Start Command:")
print("   python -m src.mcp_server.lca_mcp_server")

print("\n" + "=" * 80)

In [ ]:
# MCP Tool Call Example
print("MCP Tool Call Example: run_6agent_workflow")
print("=" * 80)

example_request = {
    "tool": "run_6agent_workflow",
    "arguments": {
        "patient_data": {
            "patient_id": "MCP_TEST_001",
            "name": "MCP_Test_Patient",
            "sex": "Female",
            "age_at_diagnosis": 68,
            "tnm_stage": "IIIA",
            "histology_type": "Adenocarcinoma",
            "performance_status": 1,
            "laterality": "Right"
        },
        "persist": False
    }
}

print("\nRequest:")
print(json.dumps(example_request, indent=2))

# Simulate the response
result = analyze_patient(example_request["arguments"]["patient_data"], persist=False)

example_response = {
    "status": "success" if result.success else "error",
    "patient_id": result.patient_id,
    "workflow_status": result.workflow_status,
    "agent_chain": result.agent_chain,
    "scenario": result.scenario,
    "scenario_confidence": result.scenario_confidence,
    "recommendations_count": len(result.recommendations),
    "processing_time_seconds": result.processing_time_seconds
}

print("\nResponse:")
print(json.dumps(example_response, indent=2))

print("\n" + "=" * 80)

---
## Part 9: Synthetic Patient Generation

Generate realistic patient cohorts for testing and validation.

In [ ]:
# Add data directory to path
sys.path.insert(0, str(Path.cwd().parent / 'data'))

try:
    from synthetic_patient_generator import SyntheticPatientGenerator
    
    print("Synthetic Patient Generation")
    print("=" * 80)
    
    # Generate synthetic patients
    generator = SyntheticPatientGenerator()
    patients = generator.generate_cohort(50)
    
    print(f"\n✓ Generated {len(patients)} synthetic patients")
    
    # Display sample
    print("\nSample Patients:")
    print("-" * 80)
    for p in patients[:5]:
        p_dict = p.to_dict() if hasattr(p, 'to_dict') else p
        print(f"   {p_dict.get('patient_id', 'N/A')}: {p_dict.get('name', 'N/A')}")
        print(f"      Stage: {p_dict.get('tnm_stage', 'N/A')}, Histology: {p_dict.get('histology_type', 'N/A')}, PS: {p_dict.get('performance_status', 'N/A')}")
    
    print(f"\n   ... and {len(patients) - 5} more")
    
except ImportError:
    print("⚠ Synthetic patient generator not available")
    print("Using built-in test patients instead.")
    
    # Create test patients manually
    import random
    
    stages = ["IA", "IB", "IIA", "IIB", "IIIA", "IIIB", "IV"]
    histologies = ["Adenocarcinoma", "SquamousCellCarcinoma", "LargeCellCarcinoma", "SmallCellCarcinoma"]
    
    patients = []
    for i in range(50):
        patients.append({
            "patient_id": f"SYNTH_{i:03d}",
            "name": f"Synthetic_Patient_{i}",
            "sex": random.choice(["M", "F"]),
            "age_at_diagnosis": random.randint(45, 85),
            "tnm_stage": random.choice(stages),
            "histology_type": random.choice(histologies),
            "performance_status": random.randint(0, 3),
            "laterality": random.choice(["Right", "Left"])
        })
    
    print(f"\n✓ Generated {len(patients)} test patients")

print("\n" + "=" * 80)

In [ ]:
# Visualize patient distribution
import matplotlib.pyplot as plt

print("Patient Cohort Distribution")
print("=" * 80)

# Convert to dictionaries if needed
patient_dicts = [p.to_dict() if hasattr(p, 'to_dict') else p for p in patients]

# Calculate distributions
stage_dist = {}
histology_dist = {}
ps_dist = {}
ages = []

for p in patient_dicts:
    stage = p.get('tnm_stage', 'Unknown')
    stage_dist[stage] = stage_dist.get(stage, 0) + 1
    
    hist = p.get('histology_type', 'Unknown')
    histology_dist[hist] = histology_dist.get(hist, 0) + 1
    
    ps = p.get('performance_status', 0)
    ps_dist[ps] = ps_dist.get(ps, 0) + 1
    
    age = p.get('age_at_diagnosis', 0)
    if age > 0:
        ages.append(age)

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Stage distribution
stage_order = ['IA', 'IB', 'IIA', 'IIB', 'IIIA', 'IIIB', 'IV']
stage_counts = [stage_dist.get(s, 0) for s in stage_order]
axes[0, 0].bar(stage_order, stage_counts, color='steelblue')
axes[0, 0].set_title('TNM Stage Distribution', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Stage')
axes[0, 0].set_ylabel('Count')

# Histology distribution
hist_labels = list(histology_dist.keys())
hist_counts = list(histology_dist.values())
axes[0, 1].barh(hist_labels, hist_counts, color='coral')
axes[0, 1].set_title('Histology Type Distribution', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Count')

# Performance Status distribution
ps_labels = [f"WHO {k}" for k in sorted(ps_dist.keys())]
ps_counts = [ps_dist[k] for k in sorted(ps_dist.keys())]
axes[1, 0].bar(ps_labels, ps_counts, color='mediumseagreen')
axes[1, 0].set_title('WHO Performance Status Distribution', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Performance Status')
axes[1, 0].set_ylabel('Count')

# Age distribution
axes[1, 1].hist(ages, bins=15, color='orchid', edgecolor='black')
axes[1, 1].set_title('Age at Diagnosis Distribution', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Age')
axes[1, 1].set_ylabel('Count')

plt.tight_layout()
plt.show()

print("\n" + "=" * 80)

---
## Part 10: Batch Processing & Cohort Analysis

In [ ]:
# Batch process patients
print("Batch Patient Processing")
print("=" * 80)

batch_results = []
batch_size = min(20, len(patients))

print(f"\nProcessing {batch_size} patients...")
print("-" * 80)

for i, patient in enumerate(patient_dicts[:batch_size]):
    try:
        result = analyze_patient(patient, persist=False)
        
        batch_results.append({
            'patient_id': patient.get('patient_id', f'UNKNOWN_{i}'),
            'stage': patient.get('tnm_stage', 'Unknown'),
            'histology': patient.get('histology_type', 'Unknown'),
            'ps': patient.get('performance_status', 0),
            'scenario': result.scenario,
            'confidence': result.scenario_confidence,
            'num_recommendations': len(result.recommendations),
            'top_treatment': result.recommendations[0]['treatment'] if result.recommendations else 'None',
            'processing_time': result.processing_time_seconds,
            'success': result.success
        })
        
        if (i + 1) % 5 == 0:
            print(f"   Processed {i + 1}/{batch_size} patients...")
            
    except Exception as e:
        print(f"   Error processing patient {patient.get('patient_id', i)}: {e}")

print(f"\n✓ Processed {len(batch_results)} patients successfully")

# Summary statistics
if batch_results:
    avg_time = sum(r['processing_time'] for r in batch_results) / len(batch_results)
    avg_recs = sum(r['num_recommendations'] for r in batch_results) / len(batch_results)
    avg_conf = sum(r['confidence'] for r in batch_results) / len(batch_results)
    
    print(f"\nBatch Statistics:")
    print(f"   Average Processing Time: {avg_time:.3f} seconds")
    print(f"   Average Recommendations: {avg_recs:.1f}")
    print(f"   Average Confidence: {avg_conf:.2%}")
    
    # Treatment distribution
    treatment_counts = {}
    for r in batch_results:
        t = r['top_treatment']
        treatment_counts[t] = treatment_counts.get(t, 0) + 1
    
    print(f"\n   Top Treatment Distribution:")
    for treatment, count in sorted(treatment_counts.items(), key=lambda x: -x[1]):
        print(f"      {treatment}: {count} ({count/len(batch_results)*100:.1f}%)")

print("\n" + "=" * 80)

In [ ]:
# Cohort Analysis Visualization
print("Cohort Analysis")
print("=" * 80)

if batch_results:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Scenario distribution
    scenario_counts = {}
    for r in batch_results:
        s = r['scenario'] or 'Unknown'
        # Shorten scenario names for display
        s_short = s.replace('_', ' ').title()[:25]
        scenario_counts[s_short] = scenario_counts.get(s_short, 0) + 1
    
    axes[0].barh(list(scenario_counts.keys()), list(scenario_counts.values()), color='teal')
    axes[0].set_title('Patient Scenario Distribution', fontweight='bold')
    axes[0].set_xlabel('Count')
    
    # Confidence distribution
    confidences = [r['confidence'] for r in batch_results if r['confidence']]
    axes[1].hist(confidences, bins=10, color='orange', edgecolor='black')
    axes[1].set_title('Classification Confidence Distribution', fontweight='bold')
    axes[1].set_xlabel('Confidence')
    axes[1].set_ylabel('Count')
    
    # Processing time distribution
    times = [r['processing_time'] for r in batch_results]
    axes[2].hist(times, bins=10, color='purple', edgecolor='black')
    axes[2].set_title('Processing Time Distribution', fontweight='bold')
    axes[2].set_xlabel('Time (seconds)')
    axes[2].set_ylabel('Count')
    
    plt.tight_layout()
    plt.show()

print("\n" + "=" * 80)

---
## Part 11: REST API Integration

The FastAPI server provides REST endpoints for external integration.

In [ ]:
print("REST API Endpoints")
print("=" * 80)

endpoints = [
    # Patient endpoints
    ("POST", "/api/v1/patients", "Create a new patient"),
    ("GET", "/api/v1/patients/{patient_id}", "Retrieve patient by ID"),
    ("POST", "/api/v1/patients/{patient_id}/analyze", "Run analysis workflow"),
    ("GET", "/api/v1/patients/{patient_id}/history", "Get treatment history"),
    
    # Treatment endpoints
    ("GET", "/api/v1/treatments/guidelines", "List all guideline rules"),
    ("POST", "/api/v1/treatments/recommend", "Generate recommendations"),
    ("GET", "/api/v1/treatments/{treatment_id}/evidence", "Get evidence details"),
    
    # Guideline endpoints
    ("GET", "/api/v1/guidelines", "List all guidelines"),
    ("GET", "/api/v1/guidelines/{rule_id}", "Get specific rule"),
    ("POST", "/api/v1/guidelines/search", "Search guidelines"),
]

print("\nAvailable Endpoints:")
print("-" * 80)

for method, path, desc in endpoints:
    print(f"   {method:6} {path}")
    print(f"          {desc}")

print("\n" + "-" * 80)
print("\nStart API Server:")
print("   uvicorn src.api.main:app --reload --port 8000")

print("\nAPI Documentation:")
print("   Swagger UI: http://localhost:8000/docs")
print("   ReDoc: http://localhost:8000/redoc")

print("\n" + "=" * 80)

In [ ]:
# API Request Example
print("API Request Example")
print("=" * 80)

example_api_request = {
    "endpoint": "POST /api/v1/patients/{patient_id}/analyze",
    "request_body": {
        "patient_id": "API_TEST_001",
        "name": "API_Test_Patient",
        "sex": "M",
        "age_at_diagnosis": 65,
        "tnm_stage": "IIIA",
        "histology_type": "Adenocarcinoma",
        "performance_status": 1
    }
}

print("\nRequest:")
print(f"   curl -X POST http://localhost:8000/api/v1/patients/API_TEST_001/analyze \\")
print(f"        -H 'Content-Type: application/json' \\")
print(f"        -d '{json.dumps(example_api_request["request_body"])}'")

print("\nExpected Response:")
print(json.dumps({
    "status": "success",
    "patient_id": "API_TEST_001",
    "scenario": "locally_advanced_unresectable",
    "recommendations": [
        {"rank": 1, "treatment": "Chemoradiotherapy", "evidence_level": "Grade A"}
    ],
    "processing_time_seconds": 0.25
}, indent=2))

print("\n" + "=" * 80)

---
## Part 12: Performance Benchmarks

In [ ]:
import time

print("Performance Benchmarks")
print("=" * 80)

# Benchmark individual agents
test_patient = {
    "patient_id": "BENCH_001",
    "name": "Benchmark_Patient",
    "sex": "Male",
    "age_at_diagnosis": 65,
    "tnm_stage": "IIIA",
    "histology_type": "Adenocarcinoma",
    "performance_status": 1,
    "laterality": "Right"
}

benchmarks = {}

# Benchmark IngestionAgent
start = time.time()
for _ in range(10):
    ingestion_agent.execute(test_patient)
benchmarks['IngestionAgent'] = (time.time() - start) / 10

# Benchmark SemanticMappingAgent
patient_fact, _ = ingestion_agent.execute(test_patient)
start = time.time()
for _ in range(10):
    semantic_agent.execute(patient_fact)
benchmarks['SemanticMappingAgent'] = (time.time() - start) / 10

# Benchmark ClassificationAgent
patient_with_codes, _ = semantic_agent.execute(patient_fact)
start = time.time()
for _ in range(10):
    classification_agent.execute(patient_with_codes)
benchmarks['ClassificationAgent'] = (time.time() - start) / 10

# Benchmark ConflictResolutionAgent
classification = classification_agent.execute(patient_with_codes)
start = time.time()
for _ in range(10):
    conflict_agent.execute(classification)
benchmarks['ConflictResolutionAgent'] = (time.time() - start) / 10

# Benchmark ExplanationAgent
resolved, _ = conflict_agent.execute(classification)
start = time.time()
for _ in range(10):
    explanation_agent.execute(patient_with_codes, resolved)
benchmarks['ExplanationAgent'] = (time.time() - start) / 10

# Benchmark complete workflow
start = time.time()
for _ in range(5):
    analyze_patient(test_patient, persist=False)
benchmarks['Complete Workflow'] = (time.time() - start) / 5

# Display results
print("\nAgent Performance (average over multiple runs):")
print("-" * 80)

for agent, time_ms in benchmarks.items():
    bar_len = int(time_ms * 100)
    bar = '█' * min(bar_len, 50)
    print(f"   {agent:25} {time_ms*1000:6.2f} ms  {bar}")

total = sum(v for k, v in benchmarks.items() if k != 'Complete Workflow')
print("-" * 80)
print(f"   {'Sum of Agents':25} {total*1000:6.2f} ms")
print(f"   {'Complete Workflow':25} {benchmarks['Complete Workflow']*1000:6.2f} ms")

print("\n" + "=" * 80)

---
## Summary

This notebook demonstrated **complete coverage** of the Lung Cancer Assistant system:

### Components Covered:

1. **SNOMED-CT Integration** - Medical terminology with 350K+ concepts
2. **LUCADA Ontology** - Domain-specific OWL ontology (60+ classes, 95+ properties)
3. **NICE CG121 Guidelines** - 7 formalized clinical rules
4. **6-Agent Workflow Architecture**:
   - IngestionAgent (validation)
   - SemanticMappingAgent (SNOMED-CT)
   - ClassificationAgent (ontology + rules)
   - ConflictResolutionAgent (ranking)
   - PersistenceAgent (Neo4j writes)
   - ExplanationAgent (MDT summaries)
5. **Neo4j Database** - Knowledge graph storage
6. **MCP Server** - 18 external integration tools
7. **REST API** - FastAPI endpoints
8. **Batch Processing** - Cohort analysis

### Key Principles:

- **"Neo4j as a tool, not a brain"** - All medical reasoning in Python/OWL
- **Strict read/write separation** - Only PersistenceAgent writes to Neo4j
- **Complete audit trail** - Every inference tracked with provenance
- **Evidence-based recommendations** - Grade A-D evidence hierarchy

### Next Steps:

- Test with real patient data (de-identified)
- Integrate with hospital EHR systems
- Deploy MCP server for clinical use
- Expand guideline rules with latest evidence
- Add outcome tracking and learning

---

**Reference:** Sesen et al., "Lung Cancer Assistant: An Ontology-Driven Clinical Decision Support System", University of Oxford